In [2]:
!git clone https://github.com/krasserm/super-resolution   #Cloning a Pretrained Model

Cloning into 'super-resolution'...
remote: Enumerating objects: 385, done.
remote: Total 385 (delta 0), reused 0 (delta 0), pack-reused 385
Receiving objects: 100% (385/385), 47.38 MiB | 8.75 MiB/s, done.
Resolving deltas: 100% (200/200), done.


In [3]:
cd /content/super-resolution #Creation of Directory

/content/super-resolution


In [4]:
!tar xvfz /content/weights-srgan.tar.gz #Loading weights

weights/srgan/gan_discriminator.h5
weights/srgan/gan_generator.h5
weights/srgan/pre_generator.h5


# **SRGAN Model Implementation :-**

---



---



In [8]:
# Importing all necessary libraries 
import timeit
import cv2 
import os
import numpy as np
from model import resolve_single
from utils import load_image, plot_sample
from model.srgan import generator

# Read the video from specified path 
cam = cv2.VideoCapture("/content/Drama144p_input.3gp") 
fps = cam.get(cv2.CAP_PROP_FPS)
print(fps)


try:
      
    # Creating a folder named data 
    if not os.path.exists('data'): 
        os.makedirs('data') 
  
# if not created then raise error 
except OSError:
    print ('Error: Creating directory of data') 
  
#frames Extraction from video 
currentframe = 0
arr_img = []
while(True): 
      
    # reading from frame 
    ret,frame = cam.read() 
  
    if ret: 
        # if video is still left continue creating images 
        name = './data/frame' + str(currentframe).zfill(3) + '.jpg'
        print ('Creating...' + name) 
  
        # writing the extracted images 
        cv2.imwrite(name, frame) 
  
        # increasing counter so that it will show how many frames are created 
        currentframe += 1
        #storing the path of extracted frames in a list
        arr_img.append(name)
    else: 
        break
#print(arr_img)

start = timeit.default_timer()
model = generator()
model.load_weights('weights/srgan/gan_generator.h5')

#Initialization of an empty list to store the super resolved images
arr_output=[]
print(len(arr_img))
n= len(arr_img)

#Implementation of SRGAN Model in extracted frames
for i in range(n):
  lr = load_image(arr_img[i])
  sr = resolve_single(model, lr)
  #plot_sample(lr, sr)
  
  arr_output.append(sr)
stop = timeit.default_timer()
#print(arr_output)

print("time : ", stop-start)

# Release all space and windows once done 
cam.release() 
cv2.destroyAllWindows()

20.0
Creating..../data/frame000.jpg
Creating..../data/frame001.jpg
Creating..../data/frame002.jpg
Creating..../data/frame003.jpg
Creating..../data/frame004.jpg
Creating..../data/frame005.jpg
Creating..../data/frame006.jpg
Creating..../data/frame007.jpg
Creating..../data/frame008.jpg
Creating..../data/frame009.jpg
Creating..../data/frame010.jpg
Creating..../data/frame011.jpg
Creating..../data/frame012.jpg
Creating..../data/frame013.jpg
Creating..../data/frame014.jpg
Creating..../data/frame015.jpg
Creating..../data/frame016.jpg
Creating..../data/frame017.jpg
Creating..../data/frame018.jpg
Creating..../data/frame019.jpg
Creating..../data/frame020.jpg
Creating..../data/frame021.jpg
Creating..../data/frame022.jpg
Creating..../data/frame023.jpg
Creating..../data/frame024.jpg
Creating..../data/frame025.jpg
Creating..../data/frame026.jpg
Creating..../data/frame027.jpg
Creating..../data/frame028.jpg
Creating..../data/frame029.jpg
Creating..../data/frame030.jpg
Creating..../data/frame031.jpg
Cre

# **Saving the Super Resolved Frames :-**
The super resolved frames are in numpy array format, so here we will change them in an image format.

In [11]:
#Importing necessary libraries
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import array_to_img
from keras.preprocessing.image import save_img

#Making a directory for storing super resolved frames in image format
os.makedirs("output_images")

#Initialization of an empty list to store the path of Super resolved frames
s_res= []
for j in range(len(arr_output)):
  out_name = '/content/super-resolution/output_images/frame' + str(j).zfill(3) + '.jpg'
  img_pil = array_to_img(arr_output[j])
  img1 = save_img(out_name, img_pil)
  s_res.append(out_name)
  
#print(s_res)

# **Conversion of Super Resolved frames into a video**

In [23]:
import cv2
import numpy as np
for i in range(len(s_res)):
    filename=s_res[i]
    #reading each files
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)

fps = 20       #Put the fps value as your convenience or 
               #Calculate by using (No. of frames)/Video_duration in seconds  

#Creation of output video               
out = cv2.VideoWriter('drama2_output.mp4',cv2.VideoWriter_fourcc(*'DIVX'), fps , size)

#Writing Frames into video
for i in range(len(s_res)):
    out.write(cv2.imread(s_res[i]))
out.release()